In [1]:
# %pip install pip -U
# %pip install pandas -U
# %pip install numpy -U
# %pip install catboost -U
# %pip install xgboost -U
# %pip install lightgbm -U
# %pip install optuna -U
# %pip install prophet -U
# %pip install mlforecast -U
# %pip install etna

In [2]:
import pandas as pd
import numpy as np
import catboost as cb
import xgboost as xgb
import lightgbm as lgb
import json
import os
from prophet import Prophet
from tqdm import tqdm
from model import Preprocessor, Optimizer, Predictor, types_changing
from config import DATA_PATH, RANDOM_STATE, models, n_folds

# Отключим предупреждения
import warnings
warnings.filterwarnings("ignore")
import logging
logger = logging.getLogger('cmdstanpy')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

/Users/freazer/Projects/lenta_hackathon/.conda/lib/python3.10/site-packages/etna/settings.py:37: UserWarning: etna[torch] is not available, to install it, run `pip install etna[torch]`
  warnings.warn("etna[torch] is not available, to install it, run `pip install etna[torch]`")
/Users/freazer/Projects/lenta_hackathon/.conda/lib/python3.10/site-packages/etna/settings.py:45: UserWarning: wandb is not available, to install it, run `pip install etna[wandb]`
  warnings.warn("wandb is not available, to install it, run `pip install etna[wandb]`")
/Users/freazer/Projects/lenta_hackathon/.conda/lib/python3.10/site-packages/etna/settings.py:62: UserWarning: etna[classification] is not available, to install it, run `pip install etna[classification]`
  warnings.warn("etna[classification] is not available, to install it, run `pip install etna[classification]`")
/Users/freazer/Projects/lenta_hackathon/.conda/lib/python3.10/site-packages/etna/settings.py:79: UserWarning: etna[statsforecast] is not av

## Загрузка и предобработка данных

In [3]:
'''В случае изменений в предобработке данных, удалить файл filled_df.csv'''
try:
    # Загрузка ранее сохраненных предобработанных данных
    with open(os.path.join(DATA_PATH, "preprocessed/cat_features.json"), "r") as file:
        CAT_FEATURES = json.load(file)
    with open(os.path.join(DATA_PATH, "preprocessed/exog_vars.json"), "r") as file:
        EXOG_VARS = json.load(file)
    filled_df = pd.read_csv(os.path.join(DATA_PATH, "preprocessed/filled_df.csv"), parse_dates=['ds'])
    types_changing(filled_df, CAT_FEATURES)
    means = pd.read_csv(os.path.join(DATA_PATH, "preprocessed/means.csv"))
except IOError:
    # Выполнение предобработки данных
    preprocessor = Preprocessor()
    filled_df, means, CAT_FEATURES, EXOG_VARS = preprocessor.train_preprocess()
    # Сохранение обработанных данных в файл
    with open(os.path.join(DATA_PATH, "preprocessed/cat_features.json"), "w") as file:
        json.dump(CAT_FEATURES, file)
    with open(os.path.join(DATA_PATH, "preprocessed/exog_vars.json"), "w") as file:
        json.dump(EXOG_VARS, file)
    filled_df.to_csv(os.path.join(DATA_PATH, "preprocessed/filled_df.csv"), index=False)
    means.to_csv(os.path.join(DATA_PATH, "preprocessed/means.csv"), index=False)

In [4]:
# filled_data = []

# for store in ['42a0e188f5033bc65bf8d78622277c4e']:#filled_df['st_id'].unique():
#         for product in tqdm(filled_df[filled_df['st_id'] == store]['unique_sku_id'].unique()):
#             subset = filled_df[(filled_df['st_id'] == store) & (filled_df['unique_sku_id'] == product)].copy()
            
#             # Получение дней недели для каждой даты
#             subset['days_of_week'] = subset['date'].dt.dayofweek

#             subset['total_sales_in_units'] = np.where(
#                 subset['total_sales_in_units'].isna(),
#                 subset.groupby('days_of_week')['total_sales_in_units'].apply(lambda x: x.rolling(4, min_periods=1).mean().shift(1)),
#                 subset['total_sales_in_units']
#                 )
            
#             subset['total_sales_in_units'].fillna(0, inplace=True)

#             # Добавляем заполненные данные в список
#             filled_data.append(subset)

# # Объединяем все заполненные данные в один DataFrame
# filled_ts = pd.concat(filled_data)

# # Сброс индекса
# filled_ts.reset_index(drop=True, inplace=True)

# # Сохранение в файл
# filled_ts.to_csv("data/filled_ts.csv", index=False)

In [5]:
groups = ['aab3238922bcc25a6f606eb525ffdc56','6512bd43d9caa6e02c990b0a82652dca','c74d97b01eae257e44aa9d5bade97baf', 
 'c51ce410c124a10e0db5e4b97fc2af39', ['c20ad4d76fe97759aa27a0c99bff6710', '32bb90e8976aab5298d5da10fe66f21d',
                                      '1ff1de774005f8da13f42943881c655f', '98f13708210194c475687be6106a3b84',
                                      '3c59dc048e8850243be8079a5c74d079']]

## Подбор гиперпараметров моделей бустингов на кросс-валидации

In [6]:
# Выборка данных для моделей. Можно заменить на любую другую выборку.
# df_test = filled_df[(filled_df['pr_group_id'] == 'aab3238922bcc25a6f606eb525ffdc56')].copy()
df_test = filled_df.loc[(filled_df['pr_sku_id'] == '00661699f543753ec7e911a64b9fd2f6')&(filled_df['st_id'] == '16a5cdae362b8d27a1d8f8c7b78b4330')].copy()

In [7]:
# Оптимизация модели
opt_res = {}
opt_res['models'] = [model.__class__.__name__ for model in models]
fold = n_folds
while fold > 0:
    file_path = os.path.join(DATA_PATH, f"optuna/{fold}_future_df.csv")
    if os.path.isfile(file_path):
        os.remove(file_path)
    fold -= 1
for model in models:
    opt = Optimizer(df_test, model, CAT_FEATURES, means)
    metric, params = opt.tuning_optimize(trials=20)
    lags = params['lags']
    del params['lags']
    opt_res[model.__class__.__name__] = ([metric, params, lags])

# Сохранение параметров в файл
with open(os.path.join(DATA_PATH, "optuna/optimization_result.json"), "w") as file:
    json.dump(opt_res, file)

[I 2023-10-09 21:57:47,317] A new study created in memory with name: no-name-c1603432-e128-47c2-adb5-b749ebe75a8e
100%|██████████| 1/1 [00:00<00:00, 45.02it/s]
[I 2023-10-09 21:57:48,619] Trial 0 finished with value: 0.3598091978739592 and parameters: {'learning_rate': 0.009432157445251358, 'max_depth': 3, 'min_child_weight': 7, 'subsample': 0.31816154783152895, 'colsample_bytree': 0.9411046606100043, 'lags': 56}. Best is trial 0 with value: 0.3598091978739592.


Trial results: [0.2999646279631604, 0.4447615139845889, 0.33470145167412824]


[I 2023-10-09 21:57:49,082] Trial 1 finished with value: 0.37565272347036266 and parameters: {'learning_rate': 0.013143470774406118, 'max_depth': 3, 'min_child_weight': 9, 'subsample': 0.9710380395998596, 'colsample_bytree': 0.18526855160446237, 'lags': 21}. Best is trial 0 with value: 0.3598091978739592.


Trial results: [0.3281867248023124, 0.4472644359723718, 0.3515070096364038]


[I 2023-10-09 21:57:49,765] Trial 2 finished with value: 0.3890157417115933 and parameters: {'learning_rate': 0.0018242190278139722, 'max_depth': 5, 'min_child_weight': 2, 'subsample': 0.8957015019945802, 'colsample_bytree': 0.906324673165299, 'lags': 49}. Best is trial 0 with value: 0.3598091978739592.


...

Trial results: [0.27574608450418486, 0.4573764609711348, 0.32259906149786743]


[I 2023-10-09 21:57:59,495] Trial 18 finished with value: 0.35214693897286803 and parameters: {'learning_rate': 0.0022473435722415343, 'max_depth': 6, 'min_child_weight': 3, 'subsample': 0.6413287608417596, 'colsample_bytree': 0.4967851866880752, 'lags': 42}. Best is trial 11 with value: 0.35099760542632863.


Trial results: [0.2748690082466117, 0.45840176333932786, 0.32317004533266447]


[I 2023-10-09 21:58:00,089] Trial 19 finished with value: 0.35315367888374505 and parameters: {'learning_rate': 0.0011695030097336454, 'max_depth': 9, 'min_child_weight': 5, 'subsample': 0.6487118177300933, 'colsample_bytree': 0.4188017495433812, 'lags': 28}. Best is trial 11 with value: 0.35099760542632863.


Trial results: [0.2712187737756147, 0.46048423166804464, 0.32775803120757574]


In [8]:
for model in opt_res['models']:
    print('Модель', model)
    # Результаты оптимизации
    print('Metric:', opt_res[model][0])
    print('Params:', opt_res[model][1])
    print('Lags:', opt_res[model][2])

    # Сохранение лучшей модели и её параметров
    best_res = 1000
    for model in opt_res['models']:
        if opt_res[model][0] < best_res:
            best_res = opt_res[model][0]
            best_model = model
    model_name = best_model
    model_params = opt_res[model][1]
    model_lags = opt_res[model][2]

Модель LGBMRegressor
Metric: 0.35099760542632863
Params: {'learning_rate': 0.00527345269253597, 'max_depth': 8, 'min_child_weight': 1, 'subsample': 0.5377281929397397, 'colsample_bytree': 0.7610880399473897}
Lags: 35


In [9]:
# Загрузка ранее сохраненных параметров моделей из файла
with open(os.path.join(DATA_PATH, "optuna/optimization_result.json"), "r") as file:
    opt_res = json.load(file)

## Получение предсказаний в будущем

In [10]:
predict_data = df_test.copy()
'''В случае изменений в предобработке данных, удалить файл future_df.csv'''
try:
    future = pd.read_csv(os.path.join(DATA_PATH, "preprocessed/future_df.csv"), parse_dates=['ds'])
    types_changing(future, CAT_FEATURES)
except IOError:
    # Выполнение предобработки данных
    preprocessor = Preprocessor(predict_data, means)
    future = preprocessor.future_prophet()
    future.to_csv(os.path.join(DATA_PATH, "preprocessed/future_df.csv"), index=False)
    # Приведение типов данных к правильным
    future = pd.read_csv(os.path.join(DATA_PATH, "preprocessed/future_df.csv"), parse_dates=['ds'])
    types_changing(future, CAT_FEATURES)

In [11]:
# Установка параметров модели
if model_name == 'CatBoostRegressor':
    del model_params['min_child_weight']
    del model_params['colsample_bytree']
    model = cb.CatBoostRegressor(**model_params, random_state=RANDOM_STATE, cat_features=CAT_FEATURES)
elif model_name == 'XGBRegressor':
    model = xgb.XGBRegressor(**model_params, random_state=RANDOM_STATE, 
                    enable_categorical=True, tree_method='hist')
elif model_name == 'LGBMRegressor':
    model = lgb.LGBMRegressor(**model_params, random_state=RANDOM_STATE)
predictor = Predictor(predict_data, future, model)

In [12]:
preds = predictor.make_predictions()
preds.rename(columns={model_name: 'total_sales_in_units'}, inplace=True)

In [13]:
preds

,unique_sku_id,ds,total_sales_in_units,st_id,pr_sku_id
0,2,2023-07-19,2.166471,16a5cdae362b8d27a1d8f8c7b78b4330,00661699f543753ec7e911a64b9fd2f6
1,2,2023-07-20,2.318248,16a5cdae362b8d27a1d8f8c7b78b4330,00661699f543753ec7e911a64b9fd2f6
2,2,2023-07-21,2.695469,16a5cdae362b8d27a1d8f8c7b78b4330,00661699f543753ec7e911a64b9fd2f6
3,2,2023-07-22,2.648785,16a5cdae362b8d27a1d8f8c7b78b4330,00661699f543753ec7e911a64b9fd2f6
4,2,2023-07-23,2.121473,16a5cdae362b8d27a1d8f8c7b78b4330,00661699f543753ec7e911a64b9fd2f6
5,2,2023-07-24,2.122470,16a5cdae362b8d27a1d8f8c7b78b4330,00661699f543753ec7e911a64b9fd2f6
6,2,2023-07-25,2.115355,16a5cdae362b8d27a1d8f8c7b78b4330,00661699f543753ec7e911a64b9fd2f6
7,2,2023-07-26,2.308588,16a5cdae362b8d27a1d8f8c7b78b4330,00661699f543753ec7e911a64b9fd2f6
8,2,2023-07-27,2.277462,16a5cdae362b8d27a1d8f8c7b78b4330,00661699f543753ec7e911a64b9fd2f6
9,2,2023-07-28,2.687093,16a5cdae362b8d27a1d8f8c7b78b4330,00661699f543753ec7e911a64b9fd2f6
